This code is for running named entity recognition (NER) using spaCy on individual Restoration-era texts that mention monopoly, East India Company, and other terms we are interested in. 
* NER: https://www.geeksforgeeks.org/python-named-entity-recognition-ner-using-spacy/
* Sorting by val: https://www.geeksforgeeks.org/python-sort-python-dictionaries-by-key-or-value/

CHALLENGES: I tried to run the algorithm on the entire monopolyFolder, but NER w/ spaCy has a max character limit of 1 million. The code ran for nearly 20 minutes before failing. What I tried next was to filter through the texts further for mentions of the East India Company to narrow down the number of texts, which resulted in 24 texts. One of the texts had more than 1 million characters, so I split it in half into two. I then ran NER on the eicFolder. I sorted the NER output by frequency and in alphabetical order. 

other notes: 
* every single Restoration monopoly EIC file (24 in total) contains some mentions of words within my small medical lexicon ("corruption|consumption|body politick|corrupt|consume|wasting|waste|blood|physician|decay|body|canker|disease|illness|remedy|cure|sickness|nervous|languish|atrophy|bleeding|bleed|melancholy|ulcer")

In [62]:
import os
import re
import pandas as pd
import string
import sys
# !{sys.executable} -m pip install spacy
# !{sys.executable} -m spacy download en_core_web_sm
import spacy

directory = r'C:\Users\amycw\Desktop\ecbc research'
monopolyFolder = directory + "\\" + 'monopoly_eebo'
tooBig = directory + "\\" + 'tooBig'
eicFolder = directory + "\\"+'eic_monopoly_eebo'

In [63]:
eicCount = 0;
medCount = 0; 

for name in os.listdir(monopolyFolder):
    readFile = pd.read_csv(monopolyFolder + "\\" + name)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        eic = re.compile("East India Company|east india company|east indies")
        if (re.search(eic, text) != None):
            moveFile = eicFolder + "\\" + name
            data.to_csv(moveFile, index=False)
            eicCount +=1
print(str(eicCount)+"\n")

for name in os.listdir(eicFolder):
    readFile = pd.read_csv(eicFolder + "\\" + name)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        lexicon = re.compile("corruption|consumption|body politick|corrupt|consume|wasting|waste|blood|physician|decay|body|canker|disease|illness|remedy|cure|sickness|nervous|languish|atrophy|bleeding|bleed|melancholy|ulcer")
        if (re.search(lexicon, text) != None):
            medCount += 1
print(str(medCount)+"\n")


24

24



In [69]:
# check for files that are bigger than 1000000 and if so, copy to another folder
for file in os.listdir(eicFolder):
    entries = pd.read_csv(eicFolder + "\\" + file)
    num_rows = entries.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = entries[i:(i+1)]
        name = tooBig + "\\" + file
        text = data.iloc[0].text
        title = data.iloc[0].title
        if (len(text)>=1000000):
            print(title+"\n")
            print(len(text))

geography rectified or a description of the world in all its kingdoms provinces countries islands cities towns seas rivers baize capes ports their ancient and present names inhabitants situations histories customs governments as also their commodities coins weights and measures compared with those at london illustrated with seventy six maps the whole work performed according to the more accurate observations and discoveries of modern authors by robert morton

1213701


In [80]:
#break the too big file into two smaller ones 

breakFile = r'C:\Users\amycw\Desktop\ecbc research\eic_monopoly_eebo\A5_P4_435.csv'
entriesOg = pd.read_csv(breakFile)
half = []
otherHalf = []
count = 0 
string = ""
other = ""

num_texts = len(entriesOg.index)
for i in range(num_texts):
    d1 = entriesOg[i:(i+1)]
    name = eicFolder + "\\" + file+"_1"
    text = d1.iloc[0].text
    words = text.split(" ")
    for w in words: 
        if (count <= (len(words)/2)):
            half.append(w)
            count+=1
        else:
            otherHalf.append(w)
for w in half:
    string = string + w + " "
for w in otherHalf: 
    other = other + w + " "

df1 = pd.DataFrame([(d1.iloc[0].index, d1.iloc[0].title,d1.iloc[0].author,d1.iloc[0].publisher,d1.iloc[0].date,string)],
           columns=('index', 'title','author','publisher','date','text'))
df2 = pd.DataFrame([(d1.iloc[0].index, d1.iloc[0].title,d1.iloc[0].author,d1.iloc[0].publisher,d1.iloc[0].date,other)],
           columns=('index', 'title','author','publisher','date','text'))
df1.to_csv(eicFolder + "\\" + 'A5_P4_435_1.csv', index=False)
df2.to_csv(eicFolder + "\\" + 'A5_P4_435_2.csv', index=False)

In [92]:
nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "a")
nerFile.write("format is (name type, frequency)\n")
ner = spacy.load('en_core_web_sm')
count=0
for fileName in os.listdir(eicFolder):
    readFile = pd.read_csv(eicFolder + "\\" + fileName)
    num_texts = len(readFile.index)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        entities = ner(text)
        for ent in entities.ents:
            nerFile.write(ent.text+" "+ent.label_+"\n")
        count+=1    
        print(count)
nerFile.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25


In [103]:
# alphabetize and sort entities by frequencies 

nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\namedEntityRecognition.txt', "r")
SortedNER = open(r'C:\Users\amycw\Desktop\ecbc research\sortedNamedEntityRecognition.txt',"a")
SortedNER.write("format is (name type, frequency)\n")
uniqueEnts = {} 
entities = []
indices = []
entities = nerFile.read().split("\n")
for ent in entities:
    if ent not in indices: 
        uniqueEnts[ent] = 1
        indices.append(ent)
    else: 
        uniqueEnts[ent] = uniqueEnts[ent] + 1 
alphabetizedEnts = {}
for ent in sorted(uniqueEnts): 
    alphabetizedEnts[ent]=uniqueEnts[ent] 
for pair in sorted (alphabetizedEnts.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    SortedNER.write(str(pair)+"\n")
SortedNER.close()

In [34]:
# failed attempt 
nerFile = open(r'C:\Users\amycw\Desktop\ecbc research\monopoly_NER.txt', "a")
nerFile.write("format is (name type, frequency)\n")

ner = spacy.load('en_core_web_sm')
uniqueEnts = {} 
for fileName in os.listdir(monopolyFolder):
    readFile = pd.read_csv(monopolyFolder + "\\" + fileName)
    num_rows = readFile.index
    num_texts = len(num_rows)
    for i in range(num_texts):
        data = readFile[i:(i+1)]
        text = data.iloc[0].text
        entities = ner(text)
        for ent in entities.ents:
            idx = ent.text+" "+ent.label_
            if ent not in uniqueEnts: 
                uniqueEnts[idx] = 1
            else: 
                uniqueEnts[idx] += 1 
alphabetizedEnts = {}
for ent in sorted(uniqueEnts): 
    alphabetizedEnts[ent]=uniqueEnts[ent] 
for pair in sorted (alphabetizedEnts.items(), key=lambda kv:(kv[1], kv[0]), reverse=True):
    nerFile.write(str(pair)+"\n")
nerFile.close()

ValueError: [E088] Text of length 1449451 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.